In [6]:
from routedConv2d import OptimizedRoutedConv2d
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

In [2]:
dataset = load_dataset("cifar10")

In [11]:
from torchvision import transforms
import torchvision

# compose the transforms
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [9]:
def collate_fn(batch):
    imgs = []
    labels = []
    for i in range(len(batch)):
        img = batch[i]["img"]
        img = transform(torchvision.transforms.ToPILImage()(img).convert("RGB"))
        imgs.append(img)
        labels.append(batch[i]["label"])
    return {
        "img": torch.stack(imgs),
        "label": torch.tensor(labels),
    }

In [2]:
import torch.nn as nn
import torch


class SparseMoEConvBlock(nn.Module):
    def __init__(
        self,
        router_dim,
        in_channels,
        num_experts=8,
        top_k=2,
        device="cuda",
    ):
        super().__init__()
        self.num_experts = num_experts
        self.in_channels = in_channels
        self.router_dim = router_dim
        self.top_k = top_k
        self.device = device

        self.router = nn.Linear(router_dim, num_experts, bias=False)
        # initialize the router weights to be random
        nn.init.xavier_uniform_(self.router.weight)
        self.experts = nn.ModuleList(
            [
                nn.Conv2d(in_channels, in_channels, 3, padding=1)
                for _ in range(num_experts)
            ]
        )

        self.register_buffer("experts_used", torch.zeros(num_experts))

    def forward(self, x):
        batch_size, _, height, width = x.shape
        router_logits = self.router(x.view(batch_size, -1))
        router_output = nn.functional.softmax(router_logits, dim=1)

        router_avg = torch.mean(router_output, dim=0)
        target = torch.tensor([1 / self.num_experts] * self.num_experts).to(self.device)
        criterion = nn.MSELoss()
        router_loss = criterion(router_avg, target)

        routing_weights, selected_experts = torch.topk(router_output, self.top_k)

        # Create mask for the selected experts (batch_size, num_experts)
        expert_mask = torch.zeros(batch_size, self.num_experts).to(x.device)
        expert_mask.scatter_(1, selected_experts, 1)

        self.experts_used = self.experts_used.to(x.device)
        self.experts_used += torch.sum(expert_mask, dim=0)

        expert_outputs = torch.zeros(
            batch_size, self.in_channels * self.top_k, height, width
        ).to(x.device)

        for i, expert_idx in enumerate(selected_experts[0]):
            expert_layer = self.experts[expert_idx]
            # We select the corresponding routing weights for the expert
            routing_weights_expert = routing_weights[0, i]

            out = expert_layer(x)
            # out = out * routing_weights_expert.unsqueeze(-1).unsqueeze(-1)

            expert_outputs[:, i * self.in_channels : (i + 1) * self.in_channels] = out

        return expert_outputs, router_loss

    def get_experts_used(self):
        return self.experts_used

    def get_experts_load_balancing_loss(self):
        return torch.std(self.experts_used)

    def reset_experts_used(self):
        self.experts_used = torch.zeros(self.num_experts)


In [3]:
class SparseMoEConvBlockWeighted(nn.Module):
    def __init__(
        self,
        router_dim,
        in_channels,
        out_channels,
        num_experts=8,
        top_k=2,
        kernel_size=3,
        stride=1,
        padding=1,
        device="cuda",
    ):
        super().__init__()
        self.num_experts = num_experts
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.router_dim = router_dim
        self.top_k = top_k
        self.device = device
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.router = nn.Linear(router_dim, num_experts, bias=False)
        # initialize the router weights to be random
        nn.init.xavier_uniform_(self.router.weight)
        self.experts = nn.ModuleList(
            [
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size=self.kernel_size,
                    stride=self.stride,
                    padding=self.padding,
                )
                for _ in range(num_experts)
            ]
        )

    def forward(self, x):
        batch_size, _, height, width = x.shape
        router_logits = self.router(x.view(batch_size, -1))
        router_output = nn.functional.softmax(router_logits, dim=1)

        router_avg = torch.mean(router_output, dim=0)
        target = torch.tensor([1 / self.num_experts] * self.num_experts).to(self.device)
        criterion = nn.MSELoss()
        router_loss = criterion(router_avg, target)

        routing_weights, selected_experts = torch.topk(router_output, self.top_k)

        expert_outputs = torch.zeros(batch_size, self.out_channels, height, width).to(
            x.device
        )

        for i, expert_idx in enumerate(selected_experts[0]):
            expert_layer = self.experts[expert_idx]
            # We select the corresponding routing weights for the expert
            routing_weights_expert = routing_weights[0, i]

            out = expert_layer(x)
            out = out * routing_weights_expert.unsqueeze(-1).unsqueeze(-1)

            expert_outputs += out

        return expert_outputs, router_loss


In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1, 1)
        self.conv2 = nn.Conv2d(6, 12, 3, 1, 1)
        self.fc1 = nn.Linear(12 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(-1, 12 * 32 * 32)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class RoutedCNN(nn.Module):
    def __init__(self, device="cuda"):
        super(RoutedCNN, self).__init__()
        self.conv1 = SparseMoEConvBlockWeighted(
            router_dim=3 * 32 * 32,
            in_channels=3,
            out_channels=24,
            num_experts=10,
            top_k=1,
        )
        # self.conv2 = SparseMoEConvBlockWeighted(
        #     router_dim=12 * 32 * 32,
        #     in_channels=12,
        #     out_channels=24,
        #     num_experts=10,
        #     top_k=1,
        # )
        self.fc1 = nn.Linear(24 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 10)
        self.device = device

    def forward(self, x, return_router_loss=False):
        x, router_loss_1 = self.conv1(x)
        x = torch.relu(x)
        # x, router_loss_2 = self.conv2(x)
        # x = torch.relu(x)
        x = x.view(-1, 24 * 32 * 32)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        if return_router_loss:
            return x, router_loss_1  # + router_loss_2
        else:
            return x


In [8]:
train_loader = DataLoader(
    dataset["train"].with_format("torch"),
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn,
)

test_loader = DataLoader(
    dataset["test"].with_format("torch"),
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn,
)

In [9]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report


num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RoutedCNN().to(device)

learning_rate = 0.0001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, batch in enumerate(tqdm(train_loader)):
        images = batch["img"].to(device)
        labels = batch["label"].to(device)

        outputs, router_loss = model(images, return_router_loss=True)
        loss = criterion(outputs, labels)

        # # Compute the load balancing loss
        # load_balancing_loss = 0
        # for module in model.modules():
        #     if isinstance(module, SparseMoEConvBlock):
        #         load_balancing_loss += module.get_experts_load_balancing_loss()
        #         module.reset_experts_used()

        # Combine the losses
        lambda_balance = (
            10  # Adjust this hyperparameter to control the strength of load balancing
        )

        total_loss = loss + lambda_balance * router_loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        if (i + 1) % len(train_loader) == 0:
            print(
                f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {total_loss.item():.4f}, Router loss: {router_loss.item():.4f}"
            )

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in tqdm(test_loader):
            images = batch["img"].to(device)
            labels = batch["label"].to(device)
            outputs = model(images)
            # for module in model.modules():
            #     if isinstance(module, SparseMoEConvBlock):
            # print(f"Experts used std: {torch.std(module.get_experts_used())}")
            # module.reset_experts_used()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(
            f"F1 score of the model on the test images: {f1_score(labels.cpu(), predicted.cpu(), average='macro')}"
        )
        print(f"Accuracy of the model on the test images: {100 * correct / total}%")


  0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [1/10], Step [1563/1563], Loss: 1.8563, Router loss: 0.0039


  0%|          | 0/313 [00:00<?, ?it/s]

F1 score of the model on the test images: 0.2
Accuracy of the model on the test images: 34.89%


  0%|          | 0/1563 [00:00<?, ?it/s]

...

F1 score of the model on the test images: 0.319047619047619
Accuracy of the model on the test images: 49.34%


  0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [10/10], Step [1563/1563], Loss: 1.5097, Router loss: 0.0085


  0%|          | 0/313 [00:00<?, ?it/s]

F1 score of the model on the test images: 0.3111111111111111
Accuracy of the model on the test images: 50.86%


In [10]:
torch.save(model.state_dict(), "routed_cnn.pth")

In [10]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    preds = []
    targets = []
    for batch in tqdm(test_loader):
        images = batch["img"].to(device)
        labels = batch["label"].to(device)
        outputs = model(images)
        # for module in model.modules():
        #     if isinstance(module, SparseMoEConvBlock):
        # print(f"Experts used std: {torch.std(module.get_experts_used())}")
        # module.reset_experts_used()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        preds.extend(predicted.cpu().numpy())
        targets.extend(labels.cpu().numpy())

    print(f"Accuracy of the model on the test images: {100 * correct / total}%")
    print(classification_report(targets, preds))

  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy of the model on the test images: 50.17%
              precision    recall  f1-score   support

           0       0.67      0.46      0.55      1000
           1       0.60      0.69      0.64      1000
           2       0.42      0.25      0.31      1000
           3       0.38      0.34      0.36      1000
           4       0.43      0.35      0.39      1000
           5       0.45      0.38      0.41      1000
           6       0.45      0.67      0.54      1000
           7       0.51      0.59      0.54      1000
           8       0.55      0.71      0.62      1000
           9       0.54      0.57      0.56      1000

    accuracy                           0.50     10000
   macro avg       0.50      0.50      0.49     10000
weighted avg       0.50      0.50      0.49     10000



In [46]:
import torch
import torch.nn as nn
import torch.optim as optim


def identify_top_experts(model, dataloader, target_class, num_top_experts=2):
    model.eval()
    expert_usage = {i: 0 for i in range(model.conv1.num_experts)}

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch["img"]
            labels = batch["label"]
            inputs, labels = inputs.to(device), labels.to(device)
            mask = labels == target_class
            if not mask.any():
                continue

            target_inputs = inputs[mask]

            # Forward pass through conv1
            router_output = model.conv1.router(
                target_inputs.view(target_inputs.size(0), -1)
            )
            _, selected_experts = torch.topk(router_output, model.conv1.top_k)

            for expert in selected_experts.flatten():
                expert_usage[expert.item()] += 1

    # Sort experts by usage and return top num_top_experts
    sorted_experts = sorted(expert_usage.items(), key=lambda x: x[1], reverse=True)
    return [expert for expert, _ in sorted_experts[:num_top_experts]]


class UnlearningLoss(nn.Module):
    def __init__(self, target_class, penalty_weight=2):
        super().__init__()
        self.target_class = target_class
        self.ce_loss = nn.CrossEntropyLoss()
        self.penalty_weight = penalty_weight

    def forward(self, outputs, labels):
        # Standard cross-entropy loss
        # change labels of target class to random class != target_class
        labels = torch.where(
            labels == self.target_class,
            torch.randint_like(labels, 0, 9),
            labels,
        )

        ce_loss = self.ce_loss(outputs, labels)

        # # Penalize correct classification of target class
        # target_mask = labels == self.target_class
        # if target_mask.any():
        #     target_outputs = outputs[target_mask]
        #     target_loss = -torch.log_softmax(target_outputs, dim=1)[
        #         :, self.target_class
        #     ].mean()
        #     return ce_loss + target_loss

        predicted = torch.argmax(outputs, 1)

        # Penalize correct classification of target class
        correct_target_class_mask = (predicted == labels) & (
            labels == self.target_class
        )

        penalty = correct_target_class_mask.sum() * self.penalty_weight

        return ce_loss + penalty


def unlearning_procedure(
    model, train_dataloader, target_class, num_epochs=20, learning_rate=0.001
):
    print(model)
    # Identify top experts for the target class
    top_experts = identify_top_experts(model, train_dataloader, target_class)
    print(f"Top experts for class {target_class}: {top_experts}")

    # Freeze all parameters except the identified experts
    for name, param in model.named_parameters():
        param.requires_grad = False

    for expert_idx in top_experts:
        for param in model.conv1.experts[expert_idx].parameters():
            param.requires_grad = True

    model.conv1.router.weight.requires_grad = True

    model.fc2.weight.requires_grad = True
    # Set up optimizer and loss function
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate
    )
    unlearning_loss = UnlearningLoss(target_class)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_dataloader:
            inputs = batch["img"]
            labels = batch["label"]
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = unlearning_loss(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(
            f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_dataloader):.4f}"
        )

    # Unfreeze all parameters
    for param in model.parameters():
        param.requires_grad = True

    print("Unlearning procedure completed.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
target_class = 1

model = RoutedCNN().to(device)
model.load_state_dict(torch.load("routed_cnn.pth"))

dataset = load_dataset("cifar10")

# train_loader = DataLoader(
#     dataset["train"].filter(lambda x: x["label"] == target_class).with_format("torch"),
#     batch_size=32,
#     shuffle=True,
#     collate_fn=collate_fn,
# )
train_loader = DataLoader(
    dataset["train"]
    .shuffle()
    .select(range(len(dataset["train"]) // 20))
    .with_format("torch"),
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn,
)

unlearning_procedure(
    model, train_loader, target_class, num_epochs=5, learning_rate=0.01
)

/tmp/ipykernel_27191/1346067593.py:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("routed_cnn.pth"))


RoutedCNN(
  (conv1): SparseMoEConvBlockWeighted(
    (router): Linear(in_features=3072, out_features=10, bias=False)
    (experts): ModuleList(
      (0-9): 10 x Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (fc1): Linear(in_features=24576, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
Top experts for class 1: [7, 1]
Epoch 1/5, Loss: 2.0120
Epoch 2/5, Loss: 1.8327
Epoch 3/5, Loss: 1.7644
Epoch 4/5, Loss: 1.8119
Epoch 5/5, Loss: 1.7596
Unlearning procedure completed.


In [47]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report


test_loader = DataLoader(
    dataset["test"].with_format("torch"),
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn,
)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    preds = []
    targets = []
    for batch in tqdm(test_loader):
        images = batch["img"].to(device)
        labels = batch["label"].to(device)
        outputs = model(images)
        # for module in model.modules():
        #     if isinstance(module, SparseMoEConvBlock):
        # print(f"Experts used std: {torch.std(module.get_experts_used())}")
        # module.reset_experts_used()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        preds.extend(predicted.cpu().numpy())
        targets.extend(labels.cpu().numpy())

    # Accuracy on target class
    correct_target = 0
    total_target = 0
    for pred, target in zip(preds, targets):
        if target == target_class:
            total_target += 1
            if pred == target:
                correct_target += 1

    # Accuracy on non-target classes
    correct_non_target = 0
    total_non_target = 0
    for pred, target in zip(preds, targets):
        if target != target_class:
            total_non_target += 1
            if pred == target:
                correct_non_target += 1
    print(
        f"Accuracy of the model on the target images: {100 * correct_target / total_target}%"
    )

    print(
        f"Accuracy of the model on the non-target images: {100 * correct_non_target / total_non_target}%"
    )
    # find indexes of target class
    target_indexes = [i for i, x in enumerate(targets) if x == target_class]

    # remove target class from predictions and targets
    preds = [pred for i, pred in enumerate(preds) if i not in target_indexes]
    targets = [target for i, target in enumerate(targets) if i not in target_indexes]

    print(f"Accuracy of the model on the test images: {100 * correct / total}%")
    print(classification_report(targets, preds, zero_division=0))
    print(
        f"F1 score of the model on the test images: {f1_score(targets, preds, average='macro')}"
    )

  0%|          | 0/313 [00:00<?, ?it/s]

Accuracy of the model on the target images: 0.1%
Accuracy of the model on the non-target images: 44.86666666666667%
Accuracy of the model on the test images: 40.39%
              precision    recall  f1-score   support

           0       0.59      0.45      0.51      1000
           2       0.35      0.26      0.30      1000
           3       0.29      0.29      0.29      1000
           4       0.31      0.47      0.37      1000
           5       0.41      0.39      0.40      1000
           6       0.40      0.69      0.51      1000
           7       0.68      0.35      0.46      1000
           8       0.59      0.64      0.61      1000
           9       0.67      0.52      0.58      1000

    accuracy                           0.45      9000
   macro avg       0.48      0.45      0.45      9000
weighted avg       0.48      0.45      0.45      9000

F1 score of the model on the test images: 0.4477874232609709
